# EcommerceSalesETL

# **ETL Pipeline for Online Retail Dataset**
This notebook implements an **ETL (Extract, Transform, Load) pipeline** for the **Online Retail Dataset** from UCI Machine Learning Repository.  

---

## **Step 1: Extract Data**
We will fetch the dataset using `ucimlrepo` and load it into a Pandas DataFrame.


In [4]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from sqlalchemy import create_engine

In [5]:
# Fetch dataset
online_retail = fetch_ucirepo(id=352)

# Load data into pandas DataFrame
X = online_retail.data.features
y = online_retail.data.targets

df = pd.concat([X, y], axis=1)  # Combine features and target if applicable

# Display basic info
df.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [7]:
# Check dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Description  540455 non-null  object 
 1   Quantity     541909 non-null  int64  
 2   InvoiceDate  541909 non-null  object 
 3   UnitPrice    541909 non-null  float64
 4   CustomerID   406829 non-null  float64
 5   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 24.8+ MB


## **Step 2: Transform Data**
We will now clean the dataset by:
- Handling missing values
- Removing duplicates
- Converting data types
- Creating new features


In [8]:
# Handling missing values
df.dropna(inplace=True)  # Drop rows with missing values

# Remove duplicates
df.drop_duplicates(inplace=True)

# Convert data types (example: date conversion)
if 'InvoiceDate' in df.columns:
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Feature Engineering: Creating a new column for total price
if 'Quantity' in df.columns and 'UnitPrice' in df.columns:
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Remove negative or zero values if necessary
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Display cleaned data
df.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [9]:
# Statistical summary
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalPrice
count,392617.000000,392617,392617.000000,392617.000000,392617.000000
mean,13.120260,2011-07-10 19:16:17.319627008,3.126076,15287.868974,22.631690
min,1.000000,2010-12-01 08:26:00,0.001000,12346.000000,0.001000
25%,2.000000,2011-04-07 11:12:00,1.250000,13955.000000,4.950000
50%,6.000000,2011-07-31 12:02:00,1.950000,15150.000000,12.450000
75%,12.000000,2011-10-20 12:53:00,3.750000,16791.000000,19.800000
max,80995.000000,2011-12-09 12:50:00,8142.750000,18287.000000,168469.600000
std,180.509714,NaN,22.243918,1713.566228,311.125487


## **Step 3: Load Data**
We will now store the cleaned data into:
1. A CSV file  
2. A SQLite database

In [10]:
# Save to CSV
df.to_csv("cleaned_online_retail.csv", index=False)

# Load into a database
engine = create_engine("sqlite:///online_retail.db")  # Using SQLite
df.to_sql("online_retail", con=engine, if_exists="replace", index=False)

print("ETL Pipeline executed successfully!")

ETL Pipeline executed successfully!
